# Naive Sentence to Emoji Translation 
## Purpose
To workshop a naive version of an sentence to emoji translation algorithm. The general idea is that sentences can be "chuncked" out into n-grams that are more related to a single emoji. The related-ness of an n-gram to an emoji is directly related to the cosine similarity of the sent2vec representation of the sentence and the sent2vec representation of one of the emoji's definitions. The emoji definitons are gathered from the [emoji2vec](https://github.com/uclmr/emoji2vec) github repo and the sent2vec model is from the [sent2vec](https://github.com/epfml/sent2vec) github repo. 

## Issues
- The generation of the summary is so incredibly slow
- There are some issues with lemmatization (e.g. poop != pooped when lemmatized)
- /opt/conda/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

## Ideas
- Add bias for fewer emojis. Some of the generated sentences are just the sentence translated into 1-grams and  it is really easy to find an emoji that represents a one word. If some how the sentence was scored both based on sum similarity and the length of the sentence that might produce better summarizations
    
- Turn the summarization into a class as to easily test new configurations of lemmatizers/stop-words. 

In [2]:
import sent2vec
from scipy.spatial.distance import cosine
from typing import List, Tuple, Callable
from itertools import combinations
import numpy as np
from nltk import word_tokenize, pos_tag
from functools import lru_cache
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
import spacy

In [3]:
# Initialize the sent2vec model
s2v = sent2vec.Sent2vecModel()
s2v.load_model('../models/wiki_unigrams.bin') # https://drive.google.com/open?id=0B6VhzidiLvjSa19uYWlLUEkzX3c

In [4]:
# Intitialize the NLTK lemmatizer
lemmatizerSpacy = spacy.load('en', disable=['parser', 'ner'])
ps = PorterStemmer()
sb = SnowballStemmer("english")
lemmatizerNLTK = WordNetLemmatizer()

## Sentence Cleaning
The general idea with sentence cleaning is that the sentences need to be put into the same "format" for better analysis. There are two main aspects of cleaning: 1) removal, and 2) modification. Removal is primarily for tokens that do not contribute to the sentence at all. These include ".", "and", "but". Normally this is a standard step in sentence cleaning but it has actually has zero effect on the output that I can see. However, token modification changes the spelling of tokens to uniform all tokens that use the same root. For example "rocked", "rock", "rocking" should all be reduced to their lemma of "rock". There are two different ways to do this: [stemming and lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html). 

In [20]:
def clean_sentence(sent: str, lemma_func: Callable[[str], str]=lemmatizerNLTK.lemmatize, keep_stop_words: bool=True) -> str:
    """
    Clean a sentence
    
    Tokenize the word and then lemmatize each individual word before rejoining it all together.
    Optionally removing stop words along the way
        
    Args:
        sent(str): Sentence to clean
        lemma_func(Callable[[str], str]): A function that takes in a word and outputs a word,
                                          normally used to pass in the lemmatization function to be mapped
                                          on every word the sentence
        keep_stop_words(bool): Keep the stop words in the sentence
    Rets:
        (str): Cleaned sentence
    """
    # Lemmatize each word in the sentence
    #return " ".join([token.lemma_ for token in lemmatizer(sent.lower())])
    return " ".join([lemma_func(token) for token in word_tokenize(sent.lower()) if token not in stopwords or keep_stop_words])

In [6]:
# Define the array to store the (emoji, repr) 2-tuple
def generate_emoji_embeddings() -> List[Tuple[str, List[float]]]:
    """
    Generate the sent2vec emoji embeddings from the input file
    
    Run each emoji within the emoji_joined data file from the emoji2vec paper through
    the sent2vec sentence embedder. This is a very naive way of doing it because one
    emoji may have multiple entries in the data file so it has multiple vectors in the
    emoji_embeddings array
    
    Args:
        None
    Rets:
        (List[Tuple[str, List[float]]]): A list of 2-tuples containing the emoji and 
                                         one vector representation of it
    """
    
    # Initialize the list that will hold all of the embedings
    emoji_embeddings = []
    
    # Open the file that stores the emoji, description 2-tuple list
    with open("emoji_joined.txt") as emojis:
        for defn in emojis:
            # The file is tab-delim
            split = defn.split("\t")

            # Get the emoji and the description from the current line
            emoji = split[-1].replace("\n", "")
            desc = clean_sentence(split[0])

            # Add each emoji and embedded description to the list
            emoji_embeddings.append((emoji, s2v.embed_sentence(desc)))
            
    # Return the embeddings
    return emoji_embeddings

emoji_embeddings = generate_emoji_embeddings()

In [7]:
@lru_cache(maxsize=100)
def closest_emoji(sent: str) -> Tuple[str, int]:
    """
    Get the closest emoji to the given sentence
    
    Loop through the list of emoji embeddings and keep track of which one has the
    lowest cosine distance from the input sentence's embedding. This is the "closest"
    emoji. The lru_cache designation means that python will store the last [maxsize]
    calls to this function with their return value to reduce computation. This is
    cleared after every call to the summary function.
    
    Args:
        sent(List[str]): Sentence to check
    Ret:
        (Tuple[str, int]) Closest emoji, cosine similarity of emoji
    
    """    
    # Embed the sentence using sent2vec 
    emb = s2v.embed_sentence(sent)

    # Start the lowest cosine at higher than it could ever be
    lowest_cos = 1_000_000

    # The best emoji starts as an empty string placeholder
    best_emoji = ""

    # Loop through the dictionary
    for emoji in emoji_embeddings:
        # Get the current emoji's embedding
        emoji_emb = emoji[1]

        # Check the cosine difference between the emoji's embedding and
        # the sentence's embedding
        curr_cos = cosine(emoji_emb, emb)

        # If it lower than the lowest then it is the new best
        if curr_cos < lowest_cos:
            lowest_cos = curr_cos
            best_emoji = emoji[0]

    # Return a 2-tuple containing the best emoji and its cosine differnece
    return best_emoji, lowest_cos

In [ ]:
def combinations_of_sent(sent: str) -> List[List[str]]:
    """
    Return all possible n-gram combinations of a sentence
    
    Args:
        sent(str): Sentence to n-gram-ify
    Rets:
        (List[List[str]]): List of all possible n-gram combinations
    """
    
    def combinations_of_sum(sum_to: int, combo: List[int]=None) -> List[List[int]]:
        """
        Return all possible combinations of ints that sum to some int
        
        Args:
            sum_to(int): The number that all sub-arrays should sum to
            combo(List[int]): The current combination of number that the recursive
                              algo should subdivide, not needed for first run but used
                              in every consequent recursive run of the function
        """
        # Initialize the list for combinations
        combos = []
        
        # If the current combo list is none (first run through)
        # then generate it with all 1s and length = sum_to
        if combo is None:
            combo = [1 for x in range(sum_to)]
            combos.append(combo)

        # Base case: If the length  of the combination is 0 then
        # end the recursion because we are at the top of the "tree"
        if len(combo) == 0:
            return None

        # For each 
        for i in range(1, len(combo)):
            combo_to_query = combo[:i-1] + [sum(combo[i - 1:i + 1])] + combo[i+1:]
            combos.append(combo_to_query)
            [combos.append(combo) for combo in combinations_of_sum(sum_to, combo_to_query) if combo is not None]

        return combos
    
    def combinations_of_sent_helper(sent):
        sent = word_tokenize(sent)
        combos = np.unique(combinations_of_sum(len(sent)))
        sent_combos = []
        for combo in combos:
            sent_combo = []
            curr_i = 0
            for combo_len in combo:
                space_joined = " ".join(sent[curr_i:combo_len + curr_i])
                if space_joined not in sent_combo:
                    sent_combo.append(space_joined) 
                curr_i += combo_len

            if sent_combo not in sent_combos:
                sent_combos.append(sent_combo)
        return sent_combos
    
    return combinations_of_sent_helper(sent)


In [3]:
def combinations_of_sum(sum_to: int, combo: List[int]=None) -> List[List[int]]:
    """
    Return all possible combinations of ints that sum to some int

    Args:
        sum_to(int): The number that all sub-arrays should sum to
        combo(List[int]): The current combination of number that the recursive
                          algo should subdivide, not needed for first run but used
                          in every consequent recursive run of the function
    """
    # Initialize the list for combinations
    combos = []

    # If the current combo list is none (first run through)
    # then generate it with all 1s and length = sum_to
    if combo is None:
        combo = [1 for x in range(sum_to)]
        combos.append(combo)

    # Base case: If the length  of the combination is 0 then
    # end the recursion because we are at the top of the "tree"
    if len(combo) == 0:
        return None

    # For each 
    for i in range(1, len(combo)):
        combo_to_query = combo[:i-1] + [sum(combo[i - 1:i + 1])] + combo[i+1:]
        combos.append(combo_to_query)
        [combos.append(combo) for combo in combinations_of_sum(sum_to, combo_to_query) if combo is not None]

    return combos

print(combinations_of_sum(10))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
def summarize(sent:str) -> Tuple[List[str], List[float], List[str]]: 
    """
    Summarize the given sentence into emojis
    
    Split the sentence into every possible combination of n-gram and see
    
    Args:
        sent(str): Sentence to summarize
    Rets:
        (Tuple[List[str], List[float], List[str]]): (Emoji Sentence, 
        List of Uncertainty values for the corresponding emoji,
        list of n-grams used to generate the corresponding emoji)
    """
    # Clean the sentence
    sent = clean_sentence(sent)
    
    # Generate all combinations of sentences
    sent_combos = combinations_of_sent(sent)
    # Init "best" datamembers as empty or exceedingly high
    best_emojis = ""
    best_n_grams = []
    best_uncertainties = [100_000_000]
    # Iterate through every combination of sentence combos
    for sent_combo in sent_combos:
        # Start the local data members as empty
        emojis = ""
        uncertainties = []
        # Iterate through each n_gram adding the uncertainty and emoji to the lists
        for n_gram in sent_combo:
            close_emoji, cos_diff = closest_emoji(n_gram)
            emojis += close_emoji
            uncertainties.append(cos_diff)

        # Check if the average uncertainty is less than the best
        # TODO: Maybe a median check would be helpful as well?
        if sum(uncertainties)/len(uncertainties) < sum(best_uncertainties)/len(best_uncertainties):
            # Update the best emojis
            best_emojis = emojis
            best_n_grams = sent_combo
            best_uncertainties = uncertainties[:]
            
    # Clear the function cache on closest_emoji because it is unlikely the next run will make use of them
    closest_emoji.cache_clear()
    
    # Return the emoji "sentence", list of all the cosine similarities, and all of the n-grams
    return (best_emojis, best_uncertainties, best_n_grams)

In [22]:
def format_summary(sents):
    emoji_embeddings = generate_emoji_embeddings()
    for sent in sents:
        summarization_res = summarize(sent)
        print(sent, "|", round(1 - sum(summarization_res[1])/len(summarization_res[1]), 3), "|", [round(x, 3) for x in summarization_res[1]] ,"|", summarization_res[2], "|", summarization_res[0] + "|")

In [23]:
sents = ["christmas music rings from the clock tower", "It not perfect but it is a start", "The sun is rising over new york city"]
format_summary(sents)

/opt/conda/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


christmas music rings from the clock tower | 0.983 | [0.0, 0.0, 0.0, 0.066] | ['christmas', 'music', 'ring', 'from the clock tower'] | 🎄🎻💍🏫|
It not perfect but it is a start | 0.879 | [0.162, 0.323, 0.0, 0.0] | ['it not', 'perfect but it is', 'a', 'start'] | 🙅💯💯🌱|
The sun is rising over new york city | 0.881 | [0.356, 0.0, 0.0] | ['the sun is rising over', 'new york', 'city'] | 🌄🗽🚏|


In [28]:
sents = ["Shrek, a mean-spirited and highly territorial green ogre", "I bet you have never seen a donkey fly!"]
format_summary(sents)

Shrek, a mean-spirited and highly territorial green ogre | 0.743 | [0.77, 0.0, 0.0] | ['shrek , a mean-spirited and highly territorial', 'green', 'ogre'] | 😾🍏👹|
I bet you have never seen a donkey fly! | 0.7 | [0.0, 0.6] | ['i', 'bet you have never seen a donkey fly !'] | 👤✈️|
